In [1]:
from dotenv import load_dotenv
import os
from pypdf import PdfReader
import gradio as gr
from openai import OpenAI

In [2]:
load_dotenv(override=True)
client = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin_text = ""
for page in reader.pages:
    linkedin_text += page.extract_text()

print(linkedin_text[:400])


   
Contact
ed.donner@gmail.com
www.linkedin.com/in/eddonner
(LinkedIn)
edwarddonner.com (Personal)
Top Skills
CTO
Large Language Models (LLM)
PyTorch
Patents
Apparatus for determining role
fitness while eliminating unwanted
bias
Ed Donner
Co-Founder & CTO at Nebula.io, repeat Co-Founder of AI startups,
speaker & advisor on Gen AI and LLM Engineering
New York, New York, United States
Summary
I’m a


In [4]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary_text = f.read()

In [5]:
name = "Ed Donner"

In [6]:
system_prompt = f"""
You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s background, experience, and skills. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so. Don't make up an answer.
"""

In [7]:
system_prompt += f"\n\n## Summary\n{summary_text}\n\n## LinkedIn Profile\n{linkedin_text}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content

gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
